Danny Abraham
<center><h1>CMPS 351</h1></center>
<center><h2>Assignment 6</h2></center>

In [2]:
import numpy as np
from numpy import linalg as la
from scipy.optimize import linprog as lp
import cvxpy as cvx
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'cvxpy'

## Center of Polyhedron
---

In [ ]:
x = cvx.Variable(2)

A = np.array( [[0., -1.], [2., -1.], [1., 1.], [-1/3, 1.], [-1., 0.], [-1., -1.]] )
b = np.array( [0., 8., 7., 3., 0., -1.] )

for i in range(len(b)):
    b[i] = b[i]/la.norm(A[i])
    A[i] = A[i]/la.norm(A[i])

In [ ]:
def radius(x):
    r = -(A@x - b)
    return r

In [ ]:
def radius_obj(x):
    #r = -(A@x - b)
    #return cvx.min(r)
    return cvx.min(r + A@x -b)

In [ ]:
obj = cvx.Maximize(radius_obj(x))

In [ ]:
def constraints_(x):
    c = []
    a = A@x
    for i,a_ in enumerate(a):
        c.append(a_ + (A[i][0]**2 + A[i][1]**2)**0.5*r <= b[i])
    return c

In [ ]:
constraints = constraints_(x)
prob = cvx.Problem(cvx.Maximize(r), constraints)
prob.solve()

In [ ]:
prob.solution

In [ ]:
x.value

In [ ]:

a = np.linspace(-5,10,10000)

fig, ax = plt.subplots()
for i in range(len(b)):
    if A[i][1] != 0:
        y = -(A[i][0]*a -b[i])/(A[i][1])
    ax.plot(y, a)
ax.plot(a, a*0)
ax.axis('equal')
circle=plt.Circle((x.value[1],x.value[0]), min(radius(x.value)), color='r')
ax.add_artist(circle)
ax.set_xlim([-2,5])
ax.set_ylim([-2,5])

In [ ]:
for c in constraints:
    print(c.dual_value)

In [ ]:
r.value

## Minimum Cost Flow
---
#### Cost Function

In [ ]:
def cost(p, x):
    return np.multiply(p,x).sum()

In [ ]:
p = np.array([[1,2],[3,4]])
x = np.array([[2,2],[3,2]])

print(cost(p, x))

In [ ]:
p = [2, 5, 3, 7, 1]
x0_bounds = [0, 5]
x1_bounds = [0, 2]
x2_bounds = [0, 1]
x3_bounds = [0, 2]
x4_bounds = [0, 4]


In [ ]:
c = p
A = [[1, 1, 0, 0, 0], 
     [-1, 0, 1, 1, 0],
     [0, -1, -1, 0, 1]]
b = [4, 0, 0]
res = lp(c, A_eq=A, b_eq=b,
         bounds=(x0_bounds, x1_bounds, x2_bounds, x3_bounds, x4_bounds),
         options = { "disp" : True } )

In [ ]:
print(res)

## Image Reconstruction
---

In [ ]:
# This template generates synthetic data (missing pixels) to use
# in testing the solution of Problem 3 in Assignment 6.
# Unknown pixels are defined by the array called "unknown".
# Obscured image is in the array U1.

#import numpy as np
#import matplotlib
#matplotlib.use("TkAgg")     # for macOS

# Read a sample image
U0 = plt.imread('bwicon.png')
m, n = U0.shape

# Create 50% mask of known pixels and use it to obscure the original
np.random.seed(7592)                 # seed the randonm number generator (for repeatability)
unknown = np.random.rand(m,n) < 0.5
U1 = U0*(1-unknown) + 0.5 *unknown


# Display images
plt.figure(1)
plt.subplot(1, 2, 1)
plt.imshow(U0, cmap='gray')
plt.title('Original image')
plt.subplot(1, 2, 2)
plt.imshow(U1, cmap='gray')
plt.title('Obscured image')
plt.show()

In [ ]:
unknown.sum()

In [ ]:
U1

In [ ]:
u = cvx.Variable(U1.shape)

In [ ]:
def cost(u):
    s = 0
    for i in range(1, U1.shape[0]):
        for j in range(1, U1.shape[1]):
            s += (u[i][j] - u[i - 1][j])**2 + (u[i][j] - u[i][j - 1])**2
    return s

In [ ]:
u[0][1]

In [ ]:
obj = cvx.Minimize(cost(u))

In [ ]:
def constraints_(u):
    c = []
    for i in range(1, U1.shape[0]):
        for j in range(1, U1.shape[1]):
            if not(unknown[i][j]):
                c.append(u[i][j] == U1[i][j])
    return c

In [ ]:
constraints = constraints_(u)

In [ ]:
prob = cvx.Problem(obj,constraints)
prob.solve()

In [ ]:
plt.imshow(u.value, cmap='gray')